In [3]:
from pyCascade import geomCreate as gm
from solid import *
import numpy as np

wthick = 0.1
# house = cube([30,10,20], False)
# house = house - translate([wthick/2, wthick/2, wthick/2])(cube(10-wthick, False))

x = 10
y = 10
z = 10
wthick = .01
nx=3
ny=1
nz=2

rooms, rooms_params = gm.makeRooms(x,y,z,wthick,nx,ny,nz)
exterior = gm.ProbedGeom(cube((x*nx,y*ny,z*nz)))
roof = gm.makeRoof((0, x*nx), (y*ny, y*(ny+1)), (0, z*nz))
doors = gm.makeDoors(rooms_params, y*.25, y*.75, 10, 10)
windows = gm.makeWindows(rooms_params, y*.25, y*.25, 10, 10)

house = exterior+roof-rooms-doors-windows

scad_render_to_file(house.geom, "./city_block_geom/OpenSCAD/test/test.scad")
print(len(house.probes))


ModuleNotFoundError: No module named 'matplotlib'

[1]

In [1]:
from pyCascade import probeSetup

# working_dir = pathlib.Path().absolute()
category =  "single_family"
run = 2
Lx = 480
n_stack = int(Lx+1)
Lx_interest = Lx/2


In [2]:
# ############ CLUSTER ################
# working_dir = '/scratch/users/nbachand/Cascade/city_block_cfd'

########### LOCAL ###################
working_dir = '/home/nbachand/Documents/Cascade/city_block'

Vertical Profiles

In [3]:
tiles = {}
x_spacings = [1,3,5]

for i in range(len(x_spacings)):
    x_dist = x_spacings[i]*(Lx/2)/6
    tiles[i] = probeSetup.y_col(x_dist,(Lx/2),[0,Lx],n_stack)
    probeSetup.writeProbes(tiles[i], f"{working_dir}/CHARLES/{category}/R{run}/probes/locations/x_{x_spacings[i]}over6.txt")
    
print('done')

done


Window Probes